Build currents on meshGrids
===============

In [1]:
import sftrident as sf
import sftrident.qft as qft
import numpy as np
import time
times = {}

Build grid of kinematic parameters
-----------

In [2]:
def BuildGrid(ss,p1x,p1y,p1m,p2x,p2y,p2m):
    SS, P1x, P1y, P1m, P2x, P2y, P2m = np.meshgrid(ss,p1x,p1y,p1m,p2x,p2y,p2m,indexing='ij')
    return [SS,P1x,P1y,P1m,P2x,P2y,P2m]

ssInit = np.array([3.353])
#p1_x = np.linspace(0.0,3.5,3)
p1_x = np.random.uniform(0.0,3.5,10)
#print "p1_x: %s"%p1_x
#p1_y = np.linspace(0.0,3.5,3)
p1_y = np.random.uniform(0.0,3.5,10)
#print "p1_y: %s"%p1_y
#p1_m = np.linspace(0.0,3.5,20)
p1_m = np.random.uniform(0.0,3.5,20)
#print "p1_m: %s"%p1_m

#p2_x = np.linspace(0.0,3.5,3)
p2_x = np.random.uniform(0.0,3.5,3)
#print "p2_x: %s"%p2_x
#p2_y = np.linspace(0.0,3.5,3)
p2_y = np.random.uniform(0.0,3.5,3)
#print "p2_y: %s"%p2_y
#p2_m = np.linspace(-1.0,3.5,20)
p2_m = np.random.uniform(-1.0,3.5,20)
#print "p2_m: %s"%p2_m
start = time.time()
kinGridRaw = BuildGrid(ssInit,p1_x,p1_y,p1_m,p2_x,p2_y,p2_m)
end = time.time() - start
times['mesh'] = end
for i,el in enumerate(kinGridRaw):
    print "coord %d: %s"%(i,str(el.shape))

coord 0: (1, 10, 10, 20, 3, 3, 20)
coord 1: (1, 10, 10, 20, 3, 3, 20)
coord 2: (1, 10, 10, 20, 3, 3, 20)
coord 3: (1, 10, 10, 20, 3, 3, 20)
coord 4: (1, 10, 10, 20, 3, 3, 20)
coord 5: (1, 10, 10, 20, 3, 3, 20)
coord 6: (1, 10, 10, 20, 3, 3, 20)


Build secondary kinematic parameters (initial particles)
----------

In [3]:
def omegaFromSS(ss):
    #lab
    return (ss**2 - 1.0)/2.0

#initial particles

start = time.time()
omega = omegaFromSS(kinGridRaw[0])
end =  time.time() -start
times['om']=end
print "omega shape: %s"%(str(omega.shape))
#print "omega: %s"%omega

start =  time.time()
E = np.ones(omega.shape)
end =  time.time() - start
times['E'] = end
print "E shape: %s"%(str(E.shape))
#print "E: %s"%E

Et = omega + E
pt = omega
print "Et shape: %s"%(str(Et.shape))
#print "Et: %s"%Et
print "pt shape: %s"%(str(pt.shape))
#print "pt: %s"%pt
start =  time.time()
#P=qft.MinkowskiVector([E,np.zeros(E.shape),np.zeros(E.shape),np.zeros(E.shape)])
P=qft.MinkowskiVector([E,0.0,0.0,0.0])
end =  time.time() - start
times['Pfirst']=end

omega shape: (1, 10, 10, 20, 3, 3, 20)
E shape: (1, 10, 10, 20, 3, 3, 20)
Et shape: (1, 10, 10, 20, 3, 3, 20)
pt shape: (1, 10, 10, 20, 3, 3, 20)


Build secondary kinematic parameters (final particles)
----------

In [4]:
start =  time.time()
P3m = 0.5*(P._0() - P._3()) - kinGridRaw[3] - kinGridRaw[6]
P3x =  - kinGridRaw[1] - kinGridRaw[4]
P3y =  - kinGridRaw[2] - kinGridRaw[5]
end =  time.time() - start
times['p3coord'] = end

print "P3m shape: %s"%(str(P3m.shape))
print "P3x shape: %s"%(str(P3x.shape))
print "P3y shape: %s"%(str(P3y.shape))

P3m shape: (1, 10, 10, 20, 3, 3, 20)
P3x shape: (1, 10, 10, 20, 3, 3, 20)
P3y shape: (1, 10, 10, 20, 3, 3, 20)


Build momenta of final particles (under condition)
------------

In [22]:
def physArea(P1m,P2m,P3m):
    return (P1m>0) * (P2m>0) * (P3m>0) 


def BuildFinalMom(px,py,pm):
    #print pm
    pp = (px**2 + py**2 + 1)/(4.0*pm)
    return qft.MinkowskiVector(qft.parray([pp+pm,px,py,pp-pm]))

def buildAllMom(p1x,p1y,p1m,p2x,p2y,p2m,p3x,p3y,p3m):
    physAreaArr = physArea(p1m,p2m,p3m)
    print "physArea: %s"%(str(physAreaArr.shape))
    #print physAreaArr.all()==False
    p1xT = p1x[physAreaArr]
    p1yT = p1y[physAreaArr]
    p1mT = p1m[physAreaArr]
    #print "p1m: %s"%(str(p1m.shape))
    #print "p1mT: %s"%(str(p1mT.shape))
    p2xT = p2x[physAreaArr]
    p2yT = p2y[physAreaArr]
    p2mT = p2m[physAreaArr]
    #print "p2m: %s"%(str(p2m.shape))
    #print "p2mT: %s"%(str(p2mT.shape))
    p3xT = p3x[physAreaArr]
    p3yT = p3y[physAreaArr]
    p3mT = p3m[physAreaArr]
    print "p3m: %s"%(str(p3m.shape))
    print "p3mT: %s"%(str(p3mT.shape))
    P1 = BuildFinalMom(p1xT,p1yT,p1mT)
    P2 = BuildFinalMom(p2xT,p2yT,p2mT)
    P3 = BuildFinalMom(p3xT,p3yT,p3mT)
    return P1,P2,P3

start =  time.time()
Pa,Pb,Pc = buildAllMom(kinGridRaw[1],kinGridRaw[2],kinGridRaw[3],kinGridRaw[4],kinGridRaw[5],kinGridRaw[6],P3x,P3y,P3m)
end =  time.time() - start
times['finalMom'] = end
#print Pa.shape
Ma = Pa*Pa
print "Mass test a: %s"%np.isclose(Ma,np.ones(Ma.shape)).all()
Mb = Pb*Pb
print "Mass test a: %s"%np.isclose(Mb,np.ones(Mb.shape)).all()
Mc = Pc*Pc
print "Mass test a: %s"%np.isclose(Mc,np.ones(Mc.shape)).all()

diffA = Ma-np.ones(Ma.shape)
print "max Diff A: %1.2e"%diffA.max()
diffB = Mb-np.ones(Mb.shape)
print "max Diff B: %1.2e"%diffB.max()
diffC = Mc-np.ones(Mc.shape)
print "max Diff C: %1.2e"%diffC.max()

physArea: (1, 10, 10, 20, 3, 3, 20)
p3m: (1, 10, 10, 20, 3, 3, 20)
p3mT: (3600,)
Mass test a: True
Mass test a: True
Mass test a: True
max Diff A: 4.55e-13
max Diff B: 2.38e-07
max Diff C: 2.91e-11


Test against serial evaluation
-------------

In [6]:
import itertools
start = time.time()
p1p3 = Pa*Pc
end = time.time() - start
print "time p1p3: %1.4e"%end
print "time p1p3: %1.4e"%(end/Pc.shape[0])
SQkinPara=[t for t in itertools.product(ssInit,p1_x,p1_y,p1_m,p2_x,p2_y,p2_m)]
ind = 0
err = 0
for i,el in enumerate(SQkinPara):
    sqP3m = 0.5 - el[3] - el[6]
    sqP3x = - el[1] - el[4]
    sqP3y = - el[2] - el[5]
    if el[3]>0 and el[6] >0 and sqP3m>0:
        sqP1 = BuildFinalMom(el[1],el[2],el[3])
        sqP2 = BuildFinalMom(el[4],el[5],el[6])
        sqP3 = BuildFinalMom(sqP3x,sqP3y,sqP3m)
        if np.isclose(sqP1(),Pa[ind]()).all():
            pass
        else:
            err+=1
            print "sq1: %s"%sqP1()
            print "bl1: %s"%Pa[ind]()
        if np.isclose(sqP2(),Pb[ind]()).all():
            pass
        else:
            err+=1
            print "sq2: %s"%sqP2()
            print "bl2: %s"%Pb[ind]()
        if np.isclose(sqP3(),Pc[ind]()).all():
            pass
        else:
            err+=1
            print "sq3: %s"%sqP3()
            print "bl3: %s"%Pc[ind]()
        
        #scalar prod test
        tp1p3 = sqP1*sqP3
        if np.isclose(tp1p3,p1p3[ind]).all():
            pass
        else:
            err+=1
            print "sq: %s"%tp1p3
            print "bl: %s"%p1p3[ind]
        
        ind +=1

print "Done with %d errors."%err

time p1p3: 3.9101e-04
time p1p3: 1.0861e-07
Done with 0 errors.


Build initial momenta
------------

In [7]:
def buildInitMom(om,e,p1x,p1y,p1m,p2x,p2y,p2m,p3x,p3y,p3m):
    physAreaArr = physArea(p1m,p2m,p3m)
    omegaT = om[physAreaArr]
    ET = e[physAreaArr]
    #P=qft.MinkowskiVector([ET,np.zeros(ET.shape),np.zeros(ET.shape),np.zeros(ET.shape)])
    P=qft.MinkowskiVector([ET,0.0,0.0,0.0])
    K = qft.MinkowskiVector([omegaT,np.zeros(omegaT.shape),np.zeros(omegaT.shape),omegaT])
    return K,P

start =  time.time()
Ktest,Ptest = buildInitMom(omega,E,kinGridRaw[1],kinGridRaw[2],kinGridRaw[3],kinGridRaw[4],kinGridRaw[5],kinGridRaw[6],P3x,P3y,P3m)
end =  time.time() - start
times['initMom'] = end
print Ktest.shape
print Ptest.shape
Mp = Ptest*Ptest
print "Mass test a: %s"%np.equal(Mp,np.ones(Mp.shape)).all()
Mk = Ktest*Ktest
print "Mass test a: %s"%np.equal(Mk,np.zeros(Mk.shape)).all()

(3600,)
(3600,)
Mass test a: True
Mass test a: True


Build Spinors (single spin)
-----------

In [8]:
gamma = qft.GammaMatrix()
g0,g1,g2,g3 =  gamma[:]

U = qft.SpinorU((Ptest,1.0),1)
UAb = qft.SpinorUBar((Pa,1.0),1)
VB = qft.SpinorV((Pb,1.0),1)
UCb = qft.SpinorUBar((Pc,1.0),1)



Test against serial evaluation
--------------

In [9]:
SQkinPara=[t for t in itertools.product(ssInit,p1_x,p1_y,p1_m,p2_x,p2_y,p2_m)]
sqP = qft.MinkowskiVector([1.0,0.0,0.0,0.0])
ind = 0
err = 0
for i,el in enumerate(SQkinPara):
    sqP3m = 0.5 - el[3] - el[6]
    sqP3x = - el[1] - el[4]
    sqP3y = - el[2] - el[5]
    if el[3]>0 and el[6] >0 and sqP3m>0:
        sqP1 = BuildFinalMom(el[1],el[2],el[3])
        sqP2 = BuildFinalMom(el[4],el[5],el[6])
        sqP3 = BuildFinalMom(sqP3x,sqP3y,sqP3m)
        
        sqU = qft.SpinorU((sqP,1.0),1)
        sqU1b = qft.SpinorUBar((sqP1,1.0),1)
        sqV2 = qft.SpinorV((sqP2,1.0),1)
        sqU3b = qft.SpinorUBar((sqP3,1.0),1)
        #print '-'*25
        #print "sqU: %s"%(type(sqU[:]))
        #print "blU: %s"%(type(U[:,ind]))
        if np.isclose(sqU[:],U[:,ind]).all():
            pass
        else:
            err+=1
        
        if np.isclose(sqU1b[:],UAb[:,ind]).all():
            pass
        else:
            err+=1
        
        if np.isclose(sqV2[:],VB[:,ind]).all():
            pass
        else:
            err+=1
        
        if np.isclose(sqU3b[:],UCb[:,ind]).all():
            pass
        else:
            err+=1
        ind +=1

print "Done with %d errors."%err

Done with 0 errors.


Build spinor products
---------

In [10]:
j10 = UAb*(g0*U)
j20 = UCb*(g0*VB)

test against serial evaluation
-------------

In [11]:
SQtimes = 0.0
start = time.time()
SQkinPara=[t for t in itertools.product(ssInit,p1_x,p1_y,p1_m,p2_x,p2_y,p2_m)]
end = time.time() - start
SQtimes += end
sqP = qft.MinkowskiVector([1.0,0.0,0.0,0.0])
ind = 0
err = 0
for i,el in enumerate(SQkinPara):
    start = time.time()
    sqP3m = 0.5 - el[3] - el[6]
    sqP3x = - el[1] - el[4]
    sqP3y = - el[2] - el[5]
    end = time.time() - start
    SQtimes += end
    if el[3]>0 and el[6] >0 and sqP3m>0:
        start = time.time()
        sqP1 = BuildFinalMom(el[1],el[2],el[3])
        sqP2 = BuildFinalMom(el[4],el[5],el[6])
        sqP3 = BuildFinalMom(sqP3x,sqP3y,sqP3m)
        
        sqU = qft.SpinorU((sqP,1.0),1)
        sqU1b = qft.SpinorUBar((sqP1,1.0),1)
        sqV2 = qft.SpinorV((sqP2,1.0),1)
        sqU3b = qft.SpinorUBar((sqP3,1.0),1)
        end = time.time() - start
        SQtimes += end
        sqU1b*(g0*sqU)
        sqU1b*(g0*sqU)
        sqU1b*(g0*sqU)
        sqU1b*(g0*sqU)
        sqU1b*(g0*sqU)
        sqU1b*(g0*sqU)
        sqU1b*(g0*sqU)
        t1 = sqU1b*(g0*sqU)
        end = time.time() - start
        SQtimes += end
        
        t2 = sqU3b*(g0*sqV2)
        
        #print '-'*25
        #print "sqT1: %s"%(t1)
        #print "blT1: %s"%(j10[ind])
        if np.isclose(t1,j10[ind]):
            pass
        else:
            err+=1
            
        if np.isclose(t2,j20[ind]):
            pass
        else:
            err+=1
            
        ind +=1

print "Done with %d errors."%err

Done with 0 errors.


Build Spinors (all spin)
----------

In [12]:
start =  time.time()
gammaMu = qft.GammaMatrix()
end =  time.time() - start
times['gamma'] = end

def buildJ0(mu,s1,s2,spinor1Bar,spinor2):
    SP1b = spinor1Bar[s1]
    SP2 = spinor2[s2]
    return SP1b*(gammaMu[mu]*SP2)

start =  time.time()
U = np.array([qft.SpinorU((Ptest,1.0),s) for s in (1,2)])
UAb = np.array([qft.SpinorUBar((Pa,1.0),1) for s in (1,2)])
VB = np.array([qft.SpinorV((Pb,1.0),1) for s in (1,2)])
UCb = np.array([qft.SpinorUBar((Pc,1.0),1) for s in (1,2)])
end =  time.time() - start
times['spinors'] = end

start = time.time()
J00 = np.array([[[buildJ0(0,s,t,UAb,U) for s in (0,1)] for t in (0,1)] for mu in np.arange(4)])
end = time.time() - start
times['buildJ00'] = end
print"%1.4e"%end

1.2149e-02


In [13]:
np.array(J00).shape

(4, 2, 2, 3600)

In [14]:
Npoints =Ktest.shape[0]
print "eval points: %d"%Npoints
print '-'*25
print "TIMES:"
print '-'*25
for el in times.iterkeys():
    print "%s: %1.4e"%(el,times[el])
    print "%s: %1.4e (avg.)"%(el,times[el]/Npoints)
    print ''
print ""
print '-'*25
print "TIMES (cummulative)"
print '-'*25
fullTime = 0.0
for el in times.iterkeys():
    fullTime+=times[el]
    print "%s: %1.4e"%(el,fullTime)
    print "%s: %1.4e (avg.)"%(el,fullTime/Npoints)
    print ''
print '-'*25
print "TIMES (properties)"
print '-'*25
maxVal = ['',0.0]

for el in times.iterkeys():
    if times[el]>=maxVal[1]:
        maxVal = [el,times[el]]
print "max: %s (%1.4e)"%(tuple(maxVal))
minVal = maxVal
for el in times.iterkeys():
    if times[el]<=minVal[1]:
        minVal = [el,times[el]]
print "min: %s (%1.4e)"%(tuple(minVal))


eval points: 3600
-------------------------
TIMES:
-------------------------
spinors: 4.8143e-02
spinors: 1.3373e-05 (avg.)

om: 3.0589e-03
om: 8.4970e-07 (avg.)

E: 1.5430e-03
E: 4.2862e-07 (avg.)

finalMom: 8.3048e-03
finalMom: 2.3069e-06 (avg.)

initMom: 6.4321e-03
initMom: 1.7867e-06 (avg.)

Pfirst: 1.9973e-02
Pfirst: 5.5481e-06 (avg.)

buildJ00: 1.2149e-02
buildJ00: 3.3747e-06 (avg.)

p3coord: 1.0803e-02
p3coord: 3.0009e-06 (avg.)

mesh: 1.0696e-02
mesh: 2.9711e-06 (avg.)

gamma: 1.7619e-04
gamma: 4.8942e-08 (avg.)


-------------------------
TIMES (cummulative)
-------------------------
spinors: 4.8143e-02
spinors: 1.3373e-05 (avg.)

om: 5.1202e-02
om: 1.4223e-05 (avg.)

E: 5.2745e-02
E: 1.4651e-05 (avg.)

finalMom: 6.1050e-02
finalMom: 1.6958e-05 (avg.)

initMom: 6.7482e-02
initMom: 1.8745e-05 (avg.)

Pfirst: 8.7455e-02
Pfirst: 2.4293e-05 (avg.)

buildJ00: 9.9604e-02
buildJ00: 2.7668e-05 (avg.)

p3coord: 1.1041e-01
p3coord: 3.0669e-05 (avg.)

mesh: 1.2110e-01
mesh: 3.3640e-05 (a

In [15]:
print "full time (ser.): %1.4e"%(SQtimes)
print "avg. time (ser.): %1.4e"%(SQtimes/Npoints)

full time (ser.): 2.0220e+01
avg. time (ser.): 5.6167e-03


In [16]:
np.__config__.show()

lapack_opt_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/lib']
    define_macros = [('HAVE_CBLAS', None)]
    language = c
blas_opt_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/lib']
    define_macros = [('HAVE_CBLAS', None)]
    language = c
openblas_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/lib']
    define_macros = [('HAVE_CBLAS', None)]
    language = c
blis_info:
  NOT AVAILABLE
openblas_lapack_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/lib']
    define_macros = [('HAVE_CBLAS', None)]
    language = c
lapack_mkl_info:
  NOT AVAILABLE
blas_mkl_info:
  NOT AVAILABLE
